In [4]:
import requests
from bs4 import BeautifulSoup
import datetime
from lxml import etree
import pandas as pd 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [5]:
# 一年中所有的日期
def get_date_list(start,end):
    start_year, start_month, start_day=start.split("/")
    end_year, end_month, end_day= end.split("/")
    begin = datetime.date(int(start_year), int(start_month), int(start_day))
    end = datetime.date(int(end_year), int(end_month), int(end_day))
    delta = datetime.timedelta(days=1)
    days = []
    while begin <= end:
        days.append(begin.strftime("%Y-%m-%d"))
        begin += delta
    date_list=list(map(lambda x: x.replace("-","/" ).replace("/","-",1), days))
    return date_list




In [6]:
def get_content(date,node_name,title,link):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
    resp = requests.get(url=link, headers=headers)
    resp.encoding = 'utf-8'
    html = etree.HTML(resp.text)
    try:
        content='\n'.join(html.xpath('//div[@id="ozoom"]//p//text()'))
    except AttributeError:
        content=''       
    return [date,node_name,title,link,content]



def news(start, end):
    date_list=get_date_list(start, end)
    #print(date_list[0:5])
    result=[]
    for date in date_list:
        url=f'http://www.macaodaily.com/html/{date}/node_1.htm'
        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
        resp=requests.get(url=url,headers=headers)
        resp.encoding='utf-8'
        html=etree.HTML(resp.text)
        #每个版块
        h_list=html.xpath('//div[@class="list"]/div/h4')
        d_list=html.xpath('//div[@class="list"]/div/div')
        for h ,d in zip(h_list,d_list):
            node_name=''.join(h.xpath('.//a/text()'))
            #print(node_name)
            li_list=d.xpath('./ul/li')
            for li in li_list:
                title=''.join(li.xpath('.//a/text()'))
                link=f'http://www.macaodaily.com/html/{date}/'+''.join(li.xpath('.//a/@href'))
                print(date,node_name,title,link)
                r=get_content(date, node_name, title, link)
                result.append(r)
    news_df=pd.DataFrame(result)
    news_df.columns=['date', 'node_name', 'title', 'link','content']
    print(news_df)
    return news_df


In [7]:

start ="2020/10/01"
end = "2020/10/02"
news_df=news(start, end)

2020-10/01 A09：要聞 李：堅定信心直面挑戰 http://www.macaodaily.com/html/2020-10/01/content_1465113.htm
2020-10/01 A09：要聞 向人民英雄敬獻花籃儀式京舉行 http://www.macaodaily.com/html/2020-10/01/content_1465114.htm
2020-10/01 A09：要聞 （社論）復興之路　　 無人能阻 http://www.macaodaily.com/html/2020-10/01/content_1465115.htm
2020-10/01 A10：經濟 餐飲推優惠  冀黃金周扳回八成生意 http://www.macaodaily.com/html/2020-10/01/content_1465116.htm
2020-10/01 A10：經濟 八月進口貨值升16% http://www.macaodaily.com/html/2020-10/01/content_1465117.htm
2020-10/01 A10：經濟 香港華人3.2億售澳門華人20%股權 http://www.macaodaily.com/html/2020-10/01/content_1465118.htm
2020-10/01 A10：經濟 內地長假航空需求料增一成 http://www.macaodaily.com/html/2020-10/01/content_1465119.htm
2020-10/01 A10：經濟 餐飲行業協會推動智慧轉型 http://www.macaodaily.com/html/2020-10/01/content_1465120.htm
2020-10/01 A10：經濟 八月新車登記少4.6% http://www.macaodaily.com/html/2020-10/01/content_1465121.htm
2020-10/01 A10：經濟 百五企參與新濠中小企學院培訓 http://www.macaodaily.com/html/2020-10/01/content_1465122.htm
2020-10/01 A10：經濟 （一家之言）培育技能 助行業結構轉型 http://www.macaodail

In [8]:
#news_df.to_excel(f"{start}至{end}".replace('/','')+'.xlsx',index=False)
#news_df.to_excel("test.xlsx",index=False)
news_df.to_pickle('./news_df-oneday.pkl')
#news_df = pd.read_csv('./news_df.pkl')

In [9]:
import tensorflow as tf
tf.__version__

'2.10.1'

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
import pandas as pd
import numpy as np
news_df = pd.read_pickle('./news_df.pkl')
news_df.head()
news_df.shape

(183720, 7)

In [2]:
news_df=news_df.replace('',np.nan)
news_df=news_df.dropna(axis=0, how='any')
news_df['node_name'].unique()
news_df['content'].shape
news_df['content'].isnull().sum()
news_df['content'].notna().sum()
news_df['content'].isna().sum()

array(['A09：要聞', 'A10：經濟', 'A11：經濟', 'A15：澳聞', 'A16：要聞', 'B01：澳聞',
       'B02：澳聞', 'B06：澳聞', 'B07：澳聞', 'B08：澳聞', 'B09：要聞', 'B12：要聞',
       'C01：體育', 'C02：體育', 'C03：澳聞', 'C04：澳聞', 'C05：澳聞', 'C06：澳聞',
       'C07：要聞', 'C08：要聞', 'C09：要聞', 'C11：澳聞', 'C12：要聞', 'D01：藝海',
       'D02：藝海', 'D03：藝海', 'D04：藝海', 'E01：醒目家', 'E04：演藝', 'E05：港聞',
       'E08：馬經', 'E09：新園地', 'E10：要聞', 'E12：樂與聲', 'A01：澳聞', 'A02：澳聞',
       'A03：澳聞', 'A04：要聞', 'A05：要聞', 'A06：澳聞', 'A07：澳聞', 'A08：要聞',
       'A12：要聞', 'B03：要聞', 'B04：要聞', 'B05：澳聞', 'B06：要聞', 'B08：港聞',
       'C03：體育', 'C04：馬經', 'C05：消閒', 'C06：航運', 'C07：新園地', 'C08：小說',
       'A01：要聞', 'B09：馬經', 'B12：馬經', 'C04：新園地', 'C05：姿采', 'C06：攝影',
       'C07：祝君健康', 'C08：姿采', 'B01：體育', 'B02：體育', 'B03：體育', 'B08：要聞',
       'B09：港聞', 'B10：馬經', 'B11：食經', 'B12：新園地', 'C01：藝海', 'C02：藝海',
       'C03：藝海', 'C04：藝海', 'C05：閱讀時間', 'C08：澳門街', 'B05：特刋', 'B09：新園地',
       'B10：要聞', 'B11：馬經', 'B12：港聞', 'C05：汽車', 'C06：汽車', 'C07：視野',
       'C08：適時識食', 'B08：圖刋', 'B11：要聞', 'C05：消費熱線',

(183706,)

0

183706

0

In [3]:
import nltk
nltk.download('stopwords')
#stopwords_list = stopwords.words('english')
from nltk.corpus import names, stopwords, words
stopwords_list = stopwords.words('chinese')

[nltk_data] Downloading package stopwords to C:\Users\STAN-
[nltk_data]     DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import string
# Take a string as input and perform the corresponding cleaning
def clean(doc): 
    # Clean up html tags
    doc = doc.replace("</br>", " ")
    # remove punctuation and numbers
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    # remove stopwords
    doc = " ".join([token for token in doc.split() if token not in stopwords_list]) 
    return doc

In [5]:
news_df['news_type'] = news_df['node_name'].apply(lambda x: x.split('：')[1])
news_df['content'] = news_df['content'].apply(lambda x: x.replace(u'\xa0', u' '))
news_df['content'] = news_df['content'].apply(clean)

In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
a=news_df['news_type'].unique()
news_df['news_type_num'] = labelencoder.fit_transform(news_df['news_type'])
b=news_df['news_type_num'].unique()
label=news_df['news_type_num'].value_counts().shape[0]

nodenum=[]
for i,j in zip(a,b):
  nodenum.append([i,j])

nodenum

[['要聞', 47],
 ['經濟', 39],
 ['澳聞', 24],
 ['體育', 63],
 ['藝海', 45],
 ['醒目家', 53],
 ['演藝', 23],
 ['港聞', 22],
 ['馬經', 61],
 ['新園地', 14],
 ['樂與聲', 17],
 ['消閒', 21],
 ['航運', 41],
 ['小說', 8],
 ['姿采', 4],
 ['攝影', 11],
 ['祝君健康', 36],
 ['食經', 59],
 ['閱讀時間', 56],
 ['澳門街', 28],
 ['特刋', 30],
 ['汽車', 18],
 ['視野', 51],
 ['適時識食', 52],
 ['圖刋', 2],
 ['消費熱線', 20],
 ['科技', 37],
 ['視覺', 50],
 ['學生報', 7],
 ['澳聞／特刋', 27],
 ['鏡海', 54],
 ['旅遊嘆世界', 16],
 ['電子競技', 57],
 ['動漫玩家', 0],
 ['蓮花廣場', 43],
 ['生活', 33],
 ['教思', 13],
 ['生產力', 34],
 ['感受澳門', 10],
 ['要聞／特刋', 49],
 ['姿采 ', 5],
 ['澳聞/特刋', 26],
 ['姿彩', 3],
 ['消聞', 19],
 ['動漫玩家 ', 1],
 ['航運 ', 42],
 ['馬經 ', 62],
 ['澳門街 ', 29],
 ['攝影 ', 12],
 ['新園地 ', 15],
 ['科技天地', 38],
 ['要聞 ', 48],
 ['姿采  ', 6],
 ['食經 ', 60],
 ['鏡海 ', 55],
 ['蓮花廣場 ', 44],
 ['藝海 ', 46],
 ['經濟 ', 40],
 ['廣告', 9],
 ['特刋（廣告）', 31],
 ['特刋／澳聞', 32],
 ['社團', 35],
 ['澳聞 / 特刋', 25],
 ['電漫玩家', 58]]

In [7]:
news_df['news_type_num'][0]
news_df['content'][0]
int_label=news_df['news_type_num'].value_counts().shape[0]
int_label

47

'國務院舉行國慶招待會 李：堅定信心直面挑戰 【據新華社北京三十日電】三十日晚，國務院在人民大會堂舉行國慶招待會，熱烈慶祝中華人民共和國成立七十一周年。習近平、李克強、栗戰書、汪洋、王滬寧、趙樂際、韓正、王岐山等黨和國家領導人與近五百位中外人士歡聚一堂，共慶共和國華誕。 人民大會堂宴會廳鮮花綻放，洋溢着熱烈喜慶的節日氣氛。主席台上方，莊嚴的國徽高懸，“－”的大字年號在鮮艷的紅旗映襯下格外醒目。 十七時三十分許，伴隨着歡快的《迎賓曲》，習近平等黨和國家領導人步入宴會廳，全場響起熱烈掌聲。 招待會開始。軍樂團奏響《義勇軍進行曲》，全場起立高唱國歌。國務院總理李克強致辭。他首先代表黨中央、國務院，向全國各族人民致以節日祝賀；向港澳同胞、台灣同胞和海外僑胞致以親切問候；向關心支持中國現代化建設事業的國際友人表示衷心感謝。 頂住壓力恢復增長 李克強說，新中國七十一年的發展史，是中國共產黨團結帶領中國人民篳路藍縷、發憤圖強的奮鬥史，是國家面貌發生巨變、人民生活水平大幅躍升的輝煌史。今年在共和國歷史上極不尋常。面對突如其來的新冠肺炎疫情和世界經濟衰退帶來的嚴重衝擊，在以習近平同志為核心的黨中央堅強領導下，我們統籌推進疫情防控和經濟社會發展，始終把人民生命安全和身體健康擺在第一位，以舉國之力與病魔搏鬥，經過艱苦努力，付出巨大犧牲，取得抗疫鬥爭重大戰略成果。做好“六穩”工作、落實“六保”任務，及時出台紓困規模性政策，創新實施機制直接惠企利民，深化改革開放，穩住經濟基本盤，頂住巨大下行壓力實現了恢復性增長，脫貧攻堅進一步取得顯著成效，人民生活得到有力保障。中國經濟展現出強勁的發展韌性，中國特色社會主義前景廣闊。 努力全面建成小康 李克強指出，當前，我國發展內外部環境發生深刻複雜變化，要堅持以習近平新時代中國特色社會主義思想為指導，立足基本國情，堅持穩中求進，堅定信心，直面挑戰，集中精力辦好自己的事。繼續做好常態化疫情防控，多措並舉助力市場主體度過難關、恢復生機，培育壯大新動能，構建新發展格局，推動高質量發展。築牢民生保障基礎，千方百計增加就業，辦好群眾關切的上學、看病、養老、托幼等方面實事，保障好受疫情、洪澇災害影響的困難群眾生活，確保打贏脫貧攻堅戰。堅定不移深化改革，更大激發市場活力和社會創造力，打造市場化、法治化、國際化營商環境，積極有為推進高水平對外開放。依靠全國上下奮力拼搏，努

64

In [8]:
condition1 = news_df['news_type']=='澳聞'
macau_news = news_df[condition1]
macau_news_content = macau_news['content']
a=macau_news_content.apply(lambda x: x.split('【')[0])
b=macau_news_content.apply(lambda x: x.split('【')[1:-1])

In [9]:
macau_news_content

19        交局：巴士財援須符指標 擬委第三方機構派神秘嘉賓評鑑乘客滿意度 【本報消息】政府已與兩巴公司...
20        兩巴：陸續更換環保車 解釋巴士埋站指引安全為首 【本報消息】新巴士合同規定巴士公司須加大環保...
21        交局：已訂系列措施迎客 【本報消息】內地已全面恢復辦理來澳自由行，今日內地開始“十 · 一”...
22        巴士漏油事故疑因壓縮泵缸破裂 當局促加強巴士保養 【本報消息】交通事務局訊：就澳門新福利公共...
23                 大排車龍 內地黃金周未到，本澳巴士已空群而出，將新馬路擠得水洩不通。 周維勤 攝
                                ...                        
183947    大橋海關檢獲化妝品鮑魚 三領隊涉借團客走水法辦 【本報消息】拱北海關所屬港珠澳大橋海關於上月...
183948    澳門航空澳國際機場參加 民航會議促增強珠澳機場功能 【本報記者廣州三十日電】民航中南地區管理...
183949    兩女網購套票遇騙失逾萬 【本報消息】網購門票詐騙頻發，近日再有兩名本地女子“中招”，一人網購...
183950    女子倒斃住所廚房死因待檢 【本報消息】昨日早上約九時許，消防及治安警接報水字巷隆盛大廈某單位...
183951    新聞特搜 危險挑戰太不應 為求流量賠性命 客流再臨市面旺 竊賊出沒慎提防 △時下流行的某款短...
Name: content, Length: 56580, dtype: object

In [10]:

from pandas.core.common import flatten
macau_news_content=list(flatten(macau_news_content))
macau_news_content=list(flatten(macau_news_content))

len(macau_news_content)

56580

In [11]:
lyrics =' '.join(row for row in macau_news_content)

In [11]:
len(lyrics)

'交局：巴士財援須符指標 擬委第三方機構派神秘嘉賓評鑑乘客滿意度 【本報消息】政府已與兩巴公司簽新合同。交通局昨指出，巴士公司需符合指標才可獲援助，否則會被扣減。巴士公司代表指出，新合同增加調度巴士靈活性，有助尖峰時段疏導乘客。亦會透過調度手段提高效率，控制成本支出。 澳門電台時事節目《澳門講場》昨探討新巴士合同，邀請交通局交通管理廳長曾祥軒、運輸管理處長何振濤，澳巴董事總經理梁美玲，以及新福利副總經理李啟見出席。 曾祥軒介紹，新巴士合同貫徹政府主導、市場運作的模式，並結合近年大衆對巴士服務意見作針對性優化。 評級越差財援越少 有聽衆不認同財政援助模式，主張經營者自負盈虧。曾祥軒表示，一九年巴士平均票價為七元一程，以現時低票價，三、四元一程公司無法抵銷成本。政府認為財政穩健是穩定服務的關鍵因素，故採取財政援助政策，並設定額封頂機制。新合同下，每班車乘車人次和總人次都要符合一九年標準，否則會有扣減機制，車廂越擠迫，則評級越差、財援越少。合同並將公衆評鑑比重由一成提高至四成，引入購買環保車等規定。 被問及新巴士合同財政援助封頂會否有財政壓力？梁美玲表示，財政壓力視乎安排，公司關注的是如何改善民生服務，如何提升市民滿意度。新合同其中一個亮點是可靈活調動車輛，現時可利用實時數據，抽調其他線路巴士，安排中途插入或特班車，讓公司能夠靈活調度疏導乘客，有助提升乘客滿意度。 李啟見稱，新合同給予公司很大靈活性，通過調度手段提高效率，以提升效率控制成本支出，保障運營效果。針對高峰期間上午七時至十時、下午四時至七時，公司會將可容納七十人的巴士換成可容納八十多人的三門巴士，增加運力。 有聽衆反映車長晚上開車太快，以恐怖形容，“扶着欄杆似做健身”。曾祥軒稱，將委托第三方學術機構負責乘客滿意度指標，派神秘嘉賓搭巴士，亦會透過民意調查，以手機App問卷方式讓市民反映乘搭感受，相信可保證評鑑公平公正，有關計劃暫時未有預算，爭取短期內有細則執行定案。 倡車廂裝錄音系統 李啟見表示，近年收到意見主要是巴士急刹和班次問題，急刹方面已使用電子監控，但無奈與電單車碰觸的事故時有發生，呼籲駕駛者保持車距，而班次則受客觀因素影響，期望可獲客觀公平的評鑑。梁美玲稱，澳巴接獲投訴會以錄像分析取證，但車長與乘客爭執取證有困難，將與交通局和個資辦溝通，在巴士車廂內安裝錄音系統，以客觀分析乘客意見。 新合同允巴士公司申

In [12]:
import tensorflow as tf

# Split the text by character and fit a vectorizer on the text in lyrics 
# Therefore, the create the mapping from words to numbers
text_vec_layer = tf.keras.layers.TextVectorization(split='character')

# adapt to the text
text_vec_layer.adapt([lyrics])

encoded = text_vec_layer([lyrics])[0]

In [13]:
import numpy as np

# inspect the vocab
vocab = np.array(text_vec_layer.get_vocabulary())
print(vocab[:20])
print(vocab[-20:])

['' '[UNK]' '，' '。' '的' ' ' '、' '澳' '會' '一' '人' '及' '有' '門' '中' '十' '年'
 '大' '作' '“']
['业' '与' '㗎' '〞' '┼' '≥' '∞' 'Ⅶ' 'Ⅵ' '℉' '̓' 'Š' 'ě' 'č' 'ā' 'ò' 'à' 'Í'
 'Ç' 'Ã']


In [14]:
# have a look at the encoded lyrics text 
encoded

<tf.Tensor: shape=(33971086,), dtype=int64, numpy=array([115,  51, 350, ..., 255,  60,   3], dtype=int64)>

In [15]:
# since we are dealing with text generation, we do not want to have empty string ''
# or [UNK] to be generated, so discard these two tokens by shifting the index 
# 2 units to the left

# drop tokens 0 (pad) and 1 (unknown)
encoded -= 2  

# number of distinct chars 
n_tokens = text_vec_layer.vocabulary_size() - 2  
dataset_size = len(encoded)  

In [16]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [17]:
# Sequence length
length = 15

In [18]:
train_index = round(dataset_size*0.8)

train_set = to_dataset(encoded[:train_index], length=length, shuffle=True)
valid_set = to_dataset(encoded[train_index:], length=length)

In [19]:
for input, target in train_set.take(1):
  print(input[:2])
  print(target[:2])

tf.Tensor(
[[ 192   29   16  371  373  571   20  153  353  152  641 2641 1397    0
    73]
 [  25  489  399  752  338  349  681    0  289  119 1173  522  181   47
    17]], shape=(2, 15), dtype=int64)
tf.Tensor(
[[  29   16  371  373  571   20  153  353  152  641 2641 1397    0   73
   269]
 [ 489  399  752  338  349  681    0  289  119 1173  522  181   47   17
    83]], shape=(2, 15), dtype=int64)


In [20]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
        tf.keras.layers.GRU(128, return_sequences=True),
        tf.keras.layers.GRU(128, return_sequences=True),
        tf.keras.layers.Dense(n_tokens, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          102192    
                                                                 
 gru (GRU)                   (None, None, 128)         56064     
                                                                 
 gru_1 (GRU)                 (None, None, 128)         99072     
                                                                 
 dense (Dense)               (None, None, 6387)        823923    
                                                                 
Total params: 1,081,251
Trainable params: 1,081,251
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile your model
model.compile(loss='sparse_categorical_crossentropy',
       optimizer='adam',
       metrics=['accuracy'])
              

history = model.fit(train_set, validation_data=valid_set, epochs=4)

Epoch 1/4
 565942/Unknown - 6356s 11ms/step - loss: 3.8147 - accuracy: 0.3209

In [32]:
song_generator_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X:X-2),
    model
])

### 4.0 - Fun time

Now it is the time to examine if your model can learn a thing or two from Jay Chou's song and write a song as good as him.

In [33]:
def next_char(model, text, temperature=1):
    y_proba = model.predict([text], verbose=0)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [34]:
def extend_text(model, text, n_chars=200, temperature=1):
    for _ in range(n_chars):
        text += next_char(model, text, temperature)
    return text

In [36]:
print(extend_text(song_generator_model, "【本報消息】", temperature=1))

【本報消息】適逢國慶內、澳門製品樂行社會議等。 打造沿岸廣場北岸的事發生工展發展的工商更獲得獎和獲“一門較正反映減時，當時更合工本來的泊廣場建設方案的爭論元，相信澳門青年會區類費會區更加問、與廣東慧一帶擠需的水客。 燈應，若有影響，但部分發揮發展格局，新合和澳門發業經濟三成協地港澳協討海區轄區海南街路巴士站及足提電統入以電單車流立改私。 澳門發展又第一入懷懷，以及居民南岸的口岸廣場的一旅客有信息“橫琴）位。 


In [ ]:
print(extend_text(song_generator_model, "一路向北", temperature=0.01))

In [ ]:
print(extend_text(song_generator_model, "一路向北", temperature=5))

In [ ]:
song_generator_model.save('./saved_model/song_gen')

In [ ]:
saved_song_gen = tf.keras.models.load_model('./saved_model/song_gen')

In [ ]:
print(extend_text(saved_song_gen, "一路向北", temperature=1))